In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers torch pandas

In [ ]:
import pandas as pd
import os
import torch
from transformers import AutoTokenizer, AutoModel
from tqdm.notebook import tqdm
import gc

# --- Configuración ---
DRIVE_PROJECT_PATH = '/content/drive/MyDrive/TFM_Cienciometria'
TMP_DIR = os.path.join(DRIVE_PROJECT_PATH, 'data/word2vec/tmp')
TMP_DIR_MATHBERT = os.path.join(DRIVE_PROJECT_PATH, 'data/mathbert/tmp')
in_csv_todo = os.path.join(TMP_DIR, "01_filtrado_abstract_ingles_TODO.csv")
out_pkl_todo = os.path.join(TMP_DIR_MATHBERT, "04_vectores_mathbert_TODO.pkl")

# --- Lógica de Vectorización OPTIMIZADA ---
def vectorize_with_mathbert(textos: list) -> list:
    device = "cuda" if torch.cuda.is_available() else "cpu"
    MODEL_NAME = "tbs17/MathBERT"

    print(f"🚀 Usando dispositivo: {device}")
    print(f"Cargando modelo: {MODEL_NAME} con precisión FP16")

    # OPTIMIZACIÓN 1: Cargar el modelo en precisión de 16 bits (FP16)
    model = AutoModel.from_pretrained(MODEL_NAME, torch_dtype=torch.float16).to(device)
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    vectores = []
    # OPTIMIZACIÓN 1.1: Aumentar el batch_size gracias al ahorro de memoria de FP16
    batch_size = 256

    print(f"Vectorizando {len(textos)} documentos con batch size = {batch_size}...")
    for i in tqdm(range(0, len(textos), batch_size)):
        batch_textos = textos[i:i + batch_size]
        inputs = tokenizer(batch_textos, padding=True, truncation=True, return_tensors="pt", max_length=512).to(device)

        with torch.no_grad():
            outputs = model(**inputs)

        attention_mask = inputs['attention_mask']
        mask_expanded = attention_mask.unsqueeze(-1).expand(outputs.last_hidden_state.size()).float()
        sum_embeddings = torch.sum(outputs.last_hidden_state * mask_expanded, 1)
        sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
        batch_vectores = sum_embeddings / sum_mask

        vectores.extend(batch_vectores.cpu().numpy())

    del model, tokenizer
    torch.cuda.empty_cache()
    return vectores

# --- Ejecución Principal ---
try:
    print(f"Cargando {in_csv_todo} (solo columnas necesarias)...")
    columnas_necesarias = ['eid', 'cover_date', 'citedby_count', 'doi', 'title', 'text']
    df = pd.read_csv(in_csv_todo, usecols=columnas_necesarias, low_memory=False)
    df["text"] = df["text"].astype(str).fillna("")

    vectores = vectorize_with_mathbert(df["text"].tolist())
    df["vector"] = vectores

    print("Preparando y guardando el archivo de salida...")
    df["cover_date"] = pd.to_datetime(df["cover_date"], errors="coerce")

    # OPTIMIZACIÓN 2: Omitir el ordenamiento si tu siguiente script ya lo hace
    # df = df.sort_values(["cover_date", "eid"]).reset_index(drop=True)

    df.to_pickle(out_pkl_todo)
    print(f"✅ ¡Proceso completado! Archivo guardado en: {out_pkl_todo}")

except Exception as e:
    print(f"❌ Ocurrió un error: {e}")

Cargando /content/drive/MyDrive/TFM_Cienciometria/data/word2vec/tmp/01_filtrado_abstract_ingles_TODO.csv (solo columnas necesarias)...
🚀 Usando dispositivo: cuda
Cargando modelo: tbs17/MathBERT con precisión FP16


config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/441M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Vectorizando 1213941 documentos con batch size = 256...


  0%|          | 0/4742 [00:00<?, ?it/s]

Preparando y guardando el archivo de salida...
✅ ¡Proceso completado! Archivo guardado en: /content/drive/MyDrive/TFM_Cienciometria/data/word2vec/tmp/04_vectores_mathbert_TODO.pkl
